# Paper's Implementation

---
#### Course: Aritificial Intelligence
#### Professor: Dr. Mehdi Ghatee
#### TA: Rouhollah Ahmadian
#### Student: Ilya Khalafi
#### Student ID: 9913039
#### December 2022 

# Table Of Contents
- [Introduction](#intro)
- [Dependencies](#dependency)
- [Dataset](#dataset)
    - [Importing Data](#import-data)
- [Preprocessing](#preprocessing)
    - [Encoding Labels](#encoding)
    - [Data Split](#split)
    - [Normalization](#normalization)
- [Paper's Implementation](#implementation)
    - [Step 1 - Determinig the number of features](#step1)
    - [Step 2 - Grouping of the features](#step2)
    - [Step 3 - Initializing particles](#step3)
    - [Step 4 - Updating the particle positions](#step4)
    - [Step 5 - Local search operations](#step5)
    - [Step 6 - Calculating fitness](#step6)
    - [Putting methods together](#final-step)
- [Final Evaluation](#evaluation)

<a name="intro"></a>

# Introduction 📚

---

This notebook implements and describes the following paper:

**Moradi, P., & Gholampour, M. (2016). A hybrid particle swarm optimization for feature subset selection by integrating a novel local search strategy. Applied Soft Computing, 43, 117–130**

Above article suggests a hybric swarm particle optimization method which has added local search to particles movement to increase the algorithm's stochasticness. The suggested algorithm as called **HPSO-LS** by the authors.

It has described the algorithm's procedure during 6 steps and we will implement the algorithm step by step. Also we will use wine dataset from sklearn library to test the algorithm and compare it to model's performance without feature selection.

<a name="dependency"></a>

#Dependencies 🧰

---

We need the following libraries during this article:

- **numpy** : <br />
    numpy is a commonly used library for doing scientific computation. Unlike python default pointer structure, numpy saves variables inplace and continous on RAM and also provides sophisticated methods that use parallelism to make our computations much faster.

- **pandas**: <br />
    pandas is also a common tool of data scientists. It provides many methods for data manipulation.

- **matplotlib** : <br />
    We will use matplotlib to show our charts.

- **scikit-learn (sklearn)** : <br />
    This library is a known data science library and we will import wine dataset from it and also some models and metric methods as well.

In [ ]:
%%capture

# Fundamental Data Analysis Tools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Importing Common useful classes and methods from scikit-learn
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder, Normalizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier

# Importing wine loader method from sklearn
from sklearn.datasets import load_wine

# We need this to supress sklearn warning messages
import warnings
from sklearn.exceptions import FitFailedWarning
warnings.filterwarnings('ignore', category=FitFailedWarning)

<a name="dataset"></a>

#Dataset ❓

---

As mentioned above, we will use wine dataset from sklearn library. Every dataset in sklearn is a **dictionary** that contains features, target, labels, dataset's description and etc.

<a name="import-data"></a>

####Importing Data

Here we load wine dataset from sklearn library.


In [ ]:
wine = load_wine()

# Every dataset in sklearn is a dictionary object
# Lets observe keys of wine dictionary
list(wine.keys())

['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names']

Now we should convert **'data'** into a pandas DataFrame with column names from **'feature_names'**.

In [ ]:
data = pd.DataFrame(wine['data'], columns = wine['feature_names'])

# Lets take a look in the imported dataset
data.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0


Also we import class labels into a single column DataFrame named **"target"**. Class labels are accessible from **'target'** key of the wine dictionary.

In [ ]:
target = pd.DataFrame(wine['target'], columns=['class'])

# Lets take a look in the imported dataset
target.head()

,class
0,0
1,0
2,0
3,0
4,0


Next we extract label of each class from **'target_names'** of the wine dictionary.

In [ ]:
list_of_labels = wine['target_names'].tolist()

labels = {i:label for i, label in enumerate(list_of_labels)}

print(labels)

{0: 'class_0', 1: 'class_1', 2: 'class_2'}


<a name="preprocessing"></a>

#Preprocessing 🧹

---

We will implement **Encoding**, **Train-Test Split** and **Normalization** steps in this section.

<a name="encoding"></a>

####Encoding Labels

Firstly we should encode class labels, we create a different column for each class and members of that class will have value of 1 in that column and other records will have a value of 0, this method of encoding is called **One Hot Encoding**.

We use **OneHotEncoder** class from sklearn library to do this task:


In [ ]:
encoded_target = pd.DataFrame(OneHotEncoder().fit_transform(target).
                              toarray().
                              astype(np.int32),
                              columns=labels.values())

encoded_target.head()

,class_0,class_1,class_2
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


<a name="split"></a>

####Data Split

We should take a proportion of the data for testing stage because model should not be evaluated by the same data that it is trained on. 

Paper mentionds that we should take 30% of the dataset for test set and the other 70% for our training set.

We use **train_test_split** method from the sklearn library:


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, encoded_target,
                                                    test_size=0.3, 
                                                    shuffle=True,
                                                    random_state=42)

<a name="normalization"></a>

####Normalization

Although we have normalized our features using **Normalizer** from sklearn library in all of the previous reports, but here we have to skip this step because the paper has suggested a custom normalization function, so we will implement the suggested method in the 6th step of the implementation...

<a name="implementation"></a>

#Implementation ⚡

---

The paper has described the algorithm in 6 steps:
 - Step 1 - Determinig the number of features
 - Step 2 - Grouping of the features
 - Step 3 - Initializing particles
 - Step 4 - Updating the particle positions
 - Step 5 - Local search operations
 - Step 6 - Calculating fitness

 Also we add a 7th step to connect the methods and define the pipeline of the algorithm.

But before the beginning, we will define to classes for our algorithm and its particles to display our road map. We will implement each method in the mentioned step.

In [ ]:
class HPSOLS:

    def random_sf(self, f):
        # Step 1
        pass

    def group_features(self, features):
        # Step 2
        pass

    def initialize_particles(num_features, k, num_particles):
        # Step 3
        pass

    def update_particles(self):
        # Step 4
        pass

    def local_search(self):
        # Step 5
        pass

class Particle:
    
    def __init__(self):
        # Step 3
        pass

    def add_features(self):
        # Step 5
        pass
    
    def delete_features(self):
        # Step 5
        pass

    def fitness(self):
        # Step 6
        pass

<a name="step1"></a>

####Step 1 - Determinig the number of features

Paper has suggested a probabilistic function to choose the number of selected features (aka. sf) which assign a weight to each possible amount of selected features. Possible values of selected features are in range [x, M * f] which f is total number of original features. Also paper suggest x = 3, notice that M < 1 and by increasing M close to 1, the sf can be chosen from bigger values, so expected value of sf increases as well.


Step 1

In [ ]:
def random_sf(self, f, x, eps):
    '''
    f = total number of original features
    x = minimum amount of selected features
    eps = epsilon defined by the paper which describes
          maximum proportion of the features that can be selected

    output : total number of features to select by the algorithm
    '''

    lsf_arr = []

    for sf in range(x, int(eps * f)):
        lsf = (f - sf) / sum(list(range(1, f-sf+1)))
        lsf_arr.append(lsf)

    total_sum = sum(lsf_arr)

    lsf_arr = [sf / total_sum for sf in lsf_arr]

    return np.random.choice(list(range(x, int(eps * f))), p=lsf_arr)

# Adding method to HPSOLS class
HPSOLS.random_sf = random_sf

<a name="step2"></a>

####Step 2 - Grouping of the features

In this step we will divide features into similar and dissimilar features. Firstly we calculate correlation among pairs of features using peasorm method and then for each features we calculate average of value of its correlation with other features. Then we sort features based on this average correlation and first half of features will make the set of dissimliar features (aka. D) and the other half will make the set of similar features (aka. S).


In [ ]:
def group_features(self, features):
    '''
    features : dataframe of features to group

    output : list D and S including dissimilar and similar features
    '''
    
    df = pd.DataFrame(features)

    correlations = df.corr(method = 'pearson')

    # We minus 1 each element of sum because i != j in cor_i calculation
    # and correlation of each feature to itself is 1.0
    cors = (correlations.abs().sum() - 1) / (len(features.columns) - 1)

    cors = cors.tolist()

    ordered = [x for _,x in sorted(zip(cors, list(range(0, len(features.columns)))))]

    D = ordered[0 : len(ordered) // 2] # group of dissimilar features
    S = ordered[len(ordered) // 2 :] # group of similar features

    return (S, D)

# Adding method to HPSOLS class
HPSOLS.group_features = group_features

<a name="step3"></a>

####Step 3 - Initializing particles

In this step we will define particle to build our swarm model. Paper defines each particles with 2 vectors, one for the position of the particle and another for the velocity, just like the ordinary PSO system. But position of each particles is a binary vector with the size of total features, so each features is selected by the particle if its element in the vector is equal to 1 and other its element will be 0. Also velocity vector has the same size as position vector and its element represent probability of each feature being selected.

For the sake of clean code, we will define particles as instances of Particle class and also pass the features and target of the training to its constructor, we will use features and target of the training set later in methods of the Particle class.

Also each particles should keep its best selected features and the value corresponding to the selected features. Step 4 of the paper has described an update formula with specific r1 and r2 values for each particles so we will keep these values for each particle as well.

In [ ]:
def initialize_particles(self, num_features, k, num_particles):
    '''
    num_features = total number of  original features, it will 
        represent size of binary vector for each particle

    k = number of selected features from step 1

    num_particles = total number of particles to initialize
    '''
    velocities = np.random.rand(num_particles, num_features)
    particles = []

    for i in range(num_particles):
        # Initializing particles position
        position = np.zeros(num_features)
        position[:k]  = 1
        np.random.shuffle(position)

        # Creating particle's object
        new_particle = Particle(position, velocities[i], self.features, self.target)
        particles.append(new_particle)
    
    return particles

# Adding method to HPSOLS class
HPSOLS.initialize_particles = initialize_particles

In [ ]:
def __particle_init__(self, selected_features, velocity, features, target):
    '''
    This method will be set as constructor of the Particle class
    '''
    self.selected_features = selected_features
    self.velocity = velocity
    self.features = features
    self.target = target
    self.r1 = np.random.random()
    self.r2 = np.random.random()
    self.best_x = self.selected_features.copy()
    self.best_x_value = self.fitness()

# Adding method to Particle class
Particle.__init__ = __particle_init__

<a name="step4"></a>

####Step 4 - Updating the particle positions

This step mostly consists of formulas. Firstly we will update each particle's velocity and then we calculate sigmoid function for each element of the velocity vector and this value shows the probability of selection for its corresponding feature, then we compare this probability with a random value in range [0, 1], if the probability was bigger that the random value we select the feature and set its value in position vector equal to 1 and 0 otherwise.

In [ ]:
def update_particles(self):
    # Step 4

    # Algorithm Parameters
    # Values are suggested by paper itself
    v_min = -4
    v_max = 4
    c1 = c2 = 2

    # best x_g is stored in instance of HPSOLS 
    best_x_g = self.best_x

    for particle in self.particles:
        # Each particle has its own r1, r2
        # absed on the formula of the Step 4
        r1 = particle.r1
        r2 = particle.r2

        # Also each particle has a presonal x_best
        best_x_i = particle.best_x

        # Updating velocity
        particle.velocity = particle.velocity + c1 * r1 * (best_x_i - particle.velocity) * np.linalg.norm(particle.velocity, 2) + c2 * r2 * (best_x_g - particle.selected_features)
        particle.velocity = np.maximum(particle.velocity, v_min)
        particle.velocity = np.minimum(particle.velocity, v_max)

        # probs define probability of each feature being selected
        # It is calculated using the sigmoid function
        probs = 1 / ( 1 + np.exp(-particle.velocity) )

        # Now we compare probability of selection for each feature
        # with a random value to decide whether to choose it or not
        particle.selected_features = (probs > np.random.rand()).astype(np.int32)

# Adding method to Particle class
HPSOLS.update_particles = update_particles

<a name="step5"></a>

####Step 5 - Local search operations

This step utilizes PSO with local search and it is the critical and key step of the paper. After updating each particles position (aka selected features) not we apply local search to adjust amount of similar and dissimilar features selected by the particle. Paper introduces parameters alpha in range [0, 1] which represents proportion of selected features that should be from similar features. We add or delete similar and dissimilar features to adjust this proportion using local search. Paper has not specified the local search so we will use hill climbing in this section.

In [ ]:
def local_search(self):
    # Step 5
    for particle in self.particles:
        # notice that features exists in D and S in ascending order according
        # to their correlation, so they will be sorted in X_d and X_s as well
        
        X_d = [feature for feature in self.D if particle.selected_features[feature] == 1]
        X_s = [feature for feature in self.S if particle.selected_features[feature] == 1]

        n_s = int(self.alpha * self.sf)
        n_d = int((1 - self.alpha) * self.sf)

        if len(X_d) < n_d:
            new_features = [feature for feature in self.D if not feature in X_d]
            particle.add_features(new_features, n_d - len(X_d))

        elif len(X_d) > n_d:
            old_features = X_d
            particle.delete_features(old_features, len(X_d) - n_d)

        if len(X_s) < n_s:
            new_features = [feature for feature in self.S if not feature in X_s]
            particle.add_features(new_features, n_s - len(X_s))

        elif len(X_s) > n_s:
            old_features = X_s
            particle.delete_features(old_features, len(X_s) - n_s)
    
        fitness = particle.fitness()

        # Updating particle's best personal position
        if fitness > particle.best_x_value:
            particle.best_x_value = fitness
            particle.best_x = particle.selected_features.copy()

        # Updating global best position
        if fitness > self.best_x_value:
            self.best_x_value = fitness
            print(f'**Updated global best value, new value: {fitness}')
            self.best_x = particle.selected_features.copy()

# Adding method to HPSOLS class
HPSOLS.local_search = local_search

In [ ]:
# We will use hill climbing for local search
def add_features(self, new_features, total_add):
    '''
    new_features : a list containing index of features
                   that can be added
    total_add : number of features to add from new_features

    output : None

    Applys local search to add 'total_add' number of features
    from list of new_features
    '''
    # Step 5
    while total_add > 0 and len(new_features) != 0:
        best_val = self.fitness()
        target_feature = -1
        # Finding best feature to add
        for feature in new_features:
            self.selected_features[feature] = 1
            val = self.fitness()
            self.selected_features[feature] = 0
            if val > best_val:
                target_feature = feature

        # If position can't be improved (local maximum)
        # Then we should return
        if target_feature == -1:
            return

        # Selecting found feature
        self.selected_features[target_feature] = 1
        new_features.remove(target_feature)
        total_add -= 1

def delete_features(self, old_features, total_delete):
    '''
    old_features : a list containing index of features
                   that can be deleted
    total_delete : number of features to delete from old_features

    output : None

    Applys local search to delete 'total_delete' number of features
    from list of old_features
    '''
    # Step 5
    while total_delete > 0 and len(old_features) != 0:
        best_val = self.fitness()
        target_feature = -1
        # Finding best feature to delete
        for feature in old_features:
            self.selected_features[feature] = 0
            val = self.fitness()
            self.selected_features[feature] = 1
            if val > best_val:
                target_feature = feature

        # If position can't be improved (local maximum)
        # Then we should return
        if target_feature == -1:
            return

        # Unselecting found feature
        self.selected_features[target_feature] = 0
        old_features.remove(target_feature)
        total_delete -= 1

# Adding method to Particle class
Particle.add_features = add_features
Particle.delete_features = delete_features

<a name="step6"></a>

####Step 6 - Calculating fitness

Last step of the paper has described the fitness method of particles. Fitness method provides a measure for selected features by the particle. Paper uses KNeighbor model to evaluate the model and also uses K Fold Validation with k = 10.

Also this step suggests a custom normalization function and we will implement it as well and inject it into instance of HPSOLS class, so to compare model's performance we will use the normalizer that exists inside the HPSOLS instance. 

In [ ]:
class CustomNormalizer:

    def fit_transform(self, features):
        self.x_max = features.max()
        self.x_min = features.min()

        return self.transform(features)

    def transform(self, features):
        # Paper mentioned that each features
        # must be in range [-1, 1]
        l = -1
        u = 1

        normal_features = l + (u-l) * (features - self.x_max) / (self.x_max - self.x_min)

        return normal_features

In [ ]:
def fitness(self):
    # Step 6
    kneighbors = KNeighborsClassifier()

    return cross_val_score(kneighbors,
                           self.features.loc[:,self.selected_features.astype(bool)],
                           self.target,
                           cv = 10, # 10 is suggested by the paper
                           n_jobs = -1,
                           error_score=0).mean()
    

# Adding method to Particle class
Particle.fitness = fitness

<a name="final-step"></a>

####Putting methods together

Now we have implemented all step and we have to put pieces of the puzzle together. Here we implement constructor method for HPSOLS class to initialize properties of the algorithm and then we define a method named **optimize** to repeat steps of the algorithm.

In [ ]:
def __init__(self, features, target, x=3, eps=0.8, num_particles=50, alpha=0.65):
    '''
    features : dataframe of features that will algorithm work on
    target : label columns for features
    x : minimum number of selecting features
    eps : maximum proportion of selecting features
    num_particles : number of particles in swarm
    alpha : proportion of selecting features to be similar

    output : this constructor method of HPSOLS class and makes
             an instance of the HPSOLS class
    '''
    self.normalizer = CustomNormalizer()
    features = self.normalizer.fit_transform(features)
    self.features = features
    self.target = target
    self.alpha = alpha
    # Step 1
    self.sf = self.random_sf(len(features.columns), x=x, eps=eps)
    # Step 2
    self.S, self.D = self.group_features(features)
    # Step 3
    self.particles = self.initialize_particles(len(features.columns), self.sf, num_particles)
    
    # Calculating global best score among all particles
    scores = [particle.best_x_value for particle in self.particles]
    positions = [particle.best_x.tolist() for particle in self.particles]
    best_pair = max( zip(scores, positions) )
    self.best_x_value = best_pair[0]
    self.best_x = np.array(best_pair[1])
    # best_x of hposls instance keeps global best score
    # but best_x of each particles just keeps its personal best

# Adding method to HPSOLS class
HPSOLS.__init__ = __init__

In [ ]:
def optimize(self, iterations=100, log_period=5):
    '''
    iterations : number of iterations to repeat algorithm procedure
    log_period : number of iteration to skip between displaying best score

    output : a numpy array which represent global best x
    '''
    for i in range(iterations):
        # Step 4
        self.update_particles()
        # Step 5
        self.local_search()

        if i % log_period == 0:
            print(f'Best value iteration {i+1}: {self.best_x_value}')

    return self.best_x

# Adding method to HPSOLS class
HPSOLS.optimize = optimize

<a name="evaluation"></a>

####Final Evaluation

Finally we start our algorithm! We will use KNeighbor Classifier to display perfomance of selected features by HPSO-LS and also we make KNeighbor Classifier to work on original features to show the difference of HPSO-LS.

In [ ]:
kneighbor = KNeighborsClassifier()

hpso = HPSOLS(X_train, y_train, x=3, eps=0.5, alpha=0.65, num_particles=5)
selected_features = hpso.optimize(iterations=20, log_period=1)

hpso_features = hpso.features.loc[:,selected_features.astype(bool)]

X_test_normal = hpso.normalizer.transform(X_test)
X_test_normal_selected = X_test_normal.loc[:, selected_features.astype(bool)]

print('-' * 30)

kneighbor.fit(hpso.features, hpso.target)
print(f'Total Features: {len(hpso.features.columns)}')
print(f'Accuracy: {kneighbor.score(X_test_normal, y_test)}')

print('-' * 30)
kneighbor.fit(hpso_features, hpso.target)
print(f'Selected Features: {len(hpso_features.columns)}')
print(f'Accuracy with HPSO-LS: {kneighbor.score(X_test_normal_selected, y_test)}')

**Updated global best value, new value: 0.9692307692307693
Best value iteration 1: 0.9692307692307693
**Updated global best value, new value: 0.9839743589743589
Best value iteration 2: 0.9839743589743589
Best value iteration 3: 0.9839743589743589
Best value iteration 4: 0.9839743589743589
Best value iteration 5: 0.9839743589743589
Best value iteration 6: 0.9839743589743589
Best value iteration 7: 0.9839743589743589
Best value iteration 8: 0.9839743589743589
Best value iteration 9: 0.9839743589743589
Best value iteration 10: 0.9839743589743589
Best value iteration 11: 0.9839743589743589
Best value iteration 12: 0.9839743589743589
Best value iteration 13: 0.9839743589743589
Best value iteration 14: 0.9839743589743589
Best value iteration 15: 0.9839743589743589
Best value iteration 16: 0.9839743589743589
Best value iteration 17: 0.9839743589743589
Best value iteration 18: 0.9839743589743589
Best value iteration 19: 0.9839743589743589
Best value iteration 20: 0.9839743589743589
-----------

We set number of particles to a small value because otherwise it would converge much faster and considering that wine dataset has only 13 features, algorithm would fine its optimal results in the first 2 iterations and that wouldn't be fun! So I sat number of particle to a small value to decrease the speed of convergence so we observer algorithms performance better!

So we see that HPSO-LS algorithm improved our model's accuracy by 6% percent and interestingly selected features helped the KNeighbor Classifier to make a perfect classification! notice that wine dataset only has 13 features so it is not a surprise that feature selection doesn't have big effect on it, but for bigger datasets with more features it is possible to see much more effect by HPSO-LS algorithm!

This notebook is available in the link below: 🙂 

https://colab.research.google.com/drive/1VqZt46HVowBmiwnacSIZKr9KCttH0kte?usp=sharing

Feel free to change HPSO-LS parameters to analyze the results...